In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [3]:
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

MODEL_NAME = "xgboost"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger(f"{ CFG[MODEL_NAME]['execution']['exp_id']}.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

2024-11-15 12:16:08.512275: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 12:16:08.973448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


exp_002
rm: cannot remove '/kaggle/output/exp_002': No such file or directory


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
set seed: 46


In [4]:
import warnings

import numpy as np
import polars as pl

warnings.filterwarnings("ignore")

In [5]:
train = pl.read_csv(os.path.join(CFG["dataset"]["competition_dir"], "train_features.csv"))
train = (
    train.with_columns(
        pl.col("ID").str.split_exact("_", n=1).struct.rename_fields(["sceneID", "offset"]).alias("fields")
    )
    .unnest("fields")
    .with_columns(pl.col("offset").cast(pl.Float32))
)
print(train.shape)
train.head(1)

(43371, 32)


ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,sceneID,offset
str,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f32
"""00066be8e20318869c38c66be46663…",5.701526,1.538456,-2.165777,-139.0,0.0,false,0.25,true,"""drive""",false,false,2.82959,0.032226,0.045187,6.231999,0.065895,0.107974,9.785009,0.124972,0.203649,13.485472,0.163448,0.302818,17.574227,0.174289,0.406331,21.951269,0.199503,0.485079,"""00066be8e20318869c38c66be46663…",320.0


## 特徴量生成


In [6]:
from penguinml.utils.contena import FeatureContena

features = FeatureContena()
features.add_num_features(
    [
        "vEgo",
        "aEgo",
        "steeringAngleDeg",
        "steeringTorque",
        "brake",
        "brakePressed",
        "gas",
        "gasPressed",
        "leftBlinker",
        "rightBlinker",
        "offset",
    ]
)
features.add_cat_features(["gearShifter"])

## ターゲット列を分解


In [7]:
from const import TARGET_COLS

train = train.unpivot(index="ID", on=TARGET_COLS, variable_name="target_name", value_name="target").join(
    train.drop(TARGET_COLS),
    on="ID",
    how="left",
)
features.add_cat_feature("target_name")
train.head(1)

ID,target_name,target,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,sceneID,offset
str,str,f64,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,str,f32
"""00066be8e20318869c38c66be46663…","""x_0""",2.82959,5.701526,1.538456,-2.165777,-139.0,0.0,false,0.25,true,"""drive""",false,false,"""00066be8e20318869c38c66be46663…",320.0


In [8]:
from const import CATEGORY_MAPPING

for c in features.num_features():
    train = train.with_columns(pl.col(c).cast(pl.Float32))

for c in features.cat_features():
    mapping = CATEGORY_MAPPING[c]
    train = train.with_columns(pl.col(c).replace_strict(mapping).cast(pl.Int32))

## CV Split


In [9]:
train_folds = pl.read_csv(CFG["dataset"]["train_fold_path"])
train = train.join(train_folds, on="sceneID", how="left")
assert train["fold"].null_count() == 0

## Training


In [10]:
from penguinml.gbdt.xgboost import fit_xgb, inference_xgb

oof, models = fit_xgb(
    data=train,
    features=features,
    params=CFG[MODEL_NAME]["params"],
    target_col="target",
    fold_col="fold",
    target_type="regression",
    verbose=500,
)

== fold 0 ==
[0]	validation_0-mae:5.36707
[500]	validation_0-mae:0.64552
[1000]	validation_0-mae:0.33295
[1500]	validation_0-mae:0.28369
[2000]	validation_0-mae:0.26632
[2500]	validation_0-mae:0.25919
[3000]	validation_0-mae:0.25609
[3500]	validation_0-mae:0.25454
[4000]	validation_0-mae:0.25310
[4500]	validation_0-mae:0.25105
[5000]	validation_0-mae:0.25036
[5500]	validation_0-mae:0.24975
[6000]	validation_0-mae:0.24935
[6500]	validation_0-mae:0.24909
[7000]	validation_0-mae:0.24890
[7500]	validation_0-mae:0.24861
[8000]	validation_0-mae:0.24817
[8500]	validation_0-mae:0.24790
[9000]	validation_0-mae:0.24768
[9500]	validation_0-mae:0.24749
[10000]	validation_0-mae:0.24727
[10500]	validation_0-mae:0.24704
[11000]	validation_0-mae:0.24677
[11500]	validation_0-mae:0.24653
[12000]	validation_0-mae:0.24628
[12500]	validation_0-mae:0.24618
[12689]	validation_0-mae:0.24618
== fold 1 ==
[0]	validation_0-mae:5.26732
[500]	validation_0-mae:0.64024
[1000]	validation_0-mae:0.32480
[1500]	validati

In [11]:
train = train.with_columns(pl.Series("oof", oof))
mae = np.abs(train["oof"] - train["target"]).mean()
print(f"MAE: {mae}")

MAE: 0.24828348654153745


In [12]:
rev_dict = {v: "oof_" + k for k, v in CATEGORY_MAPPING["target_name"].items()}
oof_df = (
    train.select(["ID", "target_name", "oof"])
    .with_columns(pl.col("target_name").replace_strict(rev_dict))
    .pivot(index="ID", columns="target_name", values="oof")
)
oof_df.head()

ID,oof_x_0,oof_y_0,oof_z_0,oof_x_1,oof_y_1,oof_z_1,oof_x_2,oof_y_2,oof_z_2,oof_x_3,oof_y_3,oof_z_3,oof_x_4,oof_y_4,oof_z_4,oof_x_5,oof_y_5,oof_z_5
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""00066be8e20318869c38c66be46663…",2.656915,0.001131,0.009627,5.893553,-0.010587,0.030023,9.18194,-0.036831,0.040837,12.965926,-0.099607,0.094276,16.847355,-0.057929,0.116801,20.483335,-0.18686,0.146723
"""00066be8e20318869c38c66be46663…",5.027689,-0.044807,0.008551,10.35842,-0.186432,0.031583,15.757709,-0.385246,0.046305,21.02219,-0.590052,0.082346,26.734488,-0.719339,0.048438,31.204971,-0.587725,0.085927
"""00066be8e20318869c38c66be46663…",4.740214,-0.001124,0.015612,9.973149,-0.036259,0.011427,15.391037,-0.088586,0.026512,21.105867,-0.172469,0.036474,26.358244,-0.23055,0.038452,31.845316,-0.236904,0.03241
"""000fb056f97572d384bae4f5fc1e0f…",2.677717,0.02272,-0.019507,5.756206,0.108209,-0.021867,8.625203,0.347694,-0.053708,11.449796,0.729984,-0.0566,14.311791,0.583957,-0.027609,16.829885,1.22661,-0.022912
"""000fb056f97572d384bae4f5fc1e0f…",1.635082,-0.028752,0.032027,3.728339,-0.162773,0.070247,6.176766,-0.300228,0.078904,8.936589,-0.53805,0.103499,11.993236,-1.031518,0.151469,15.168142,-0.66102,0.173373


In [13]:
oof_df.write_csv(os.path.join(CFG["output_dir"], "oof.csv"))